# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _ALIFE Phylogeny Tutorial Notebook_

[Link to original julia notebook template](https://github.com/ageron/julia_notebooks/blob/main/Julia_Colab_Notebook_Template.ipynb)



# Installation

## Install Julia
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.10.4" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=8
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.10.4 on the current Colab Runtime...
2024-07-03 14:50:02 URL:https://julialang-s3.julialang.org/bin/linux/x64/1.10/julia-1.10.4-linux-x86_64.tar.gz [173704015/173704015] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.10

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


## Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [ ]:
versioninfo()

Julia Version 1.10.4
Commit 48d4fd48430 (2024-06-04 10:41 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, broadwell)
Threads: 8 default, 0 interactive, 2 GC (on 2 virtual cores)
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = 8


## Install Jevo.jl and Dependencies

Run the following cell to install the `phylo-tutorial` branch of [Jevo.jl](https://www.github.com/jarbus/Jevo.jl) and its dependencies. This branch has all of the neuro-evolution code stripped out to install faster.

[Jevo.jl](https://www.github.com/jarbus/Jevo.jl) is currently in alpha, development docs can be found [here](https://jarbus.github.io/Jevo.jl/dev)

This cell should take ~10 minutes on Google Collab.

In [ ]:
]add https://github.com/jarbus/XPlot.jl.git https://github.com/jarbus/PhylogeneticTrees.jl.git https://github.com/jarbus/Jevo.jl.git#phylo-tutorial StableRNGs Logging

    Updating git-repo `https://github.com/jarbus/XPlot.jl.git`
    Updating git-repo `https://github.com/jarbus/PhylogeneticTrees.jl.git`
    Updating git-repo `https://github.com/jarbus/Jevo.jl.git`
   Resolving package versions...
    Updating `~/.julia/environments/v1.10/Project.toml`
  [860ef19b] + StableRNGs v1.0.2
  [56ddb016] + Logging
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
Precompiling project...
  ✓ Accessors → AccessorsUnitfulExt
  1 dependency successfully precompiled in 1 seconds. 229 already precompiled.


# Run a Co-evolutionary simulation

In [ ]:
using Jevo
using Logging
using StableRNGs

STATS_FILE = "statistics.h5"
isfile(STATS_FILE) && rm(STATS_FILE)

global_logger(JevoLogger())
rng = StableRNG(1)

k = 1
n_dims = 2
n_inds = 2
n_species = 2
n_gens = 10

counters = default_counters()
ng_gc = ng_genotype_creator = Creator(VectorGenotype, (n=n_dims,rng=rng))
ng_developer = Creator(VectorPhenotype)

comp_pop_creator = Creator(CompositePopulation, ("species", [("p$i", n_inds, ng_gc, ng_developer) for i in 1:n_species], counters))
env_creator = Creator(CompareOnOne)

#
state = State("numbers_game", rng,
    # State accepts a list of creators
    [comp_pop_creator, env_creator],
    # and a sequence of operators to apply to the state
    [InitializeAllPopulations(),
    InitializePhylogeny(),
    AllVsAllMatchMaker(),
    Performer(),
    ScalarFitnessEvaluator(),
    TruncationSelector(k),
    CloneUniformReproducer(n_inds),
    Mutator(),
    UpdatePhylogeny(),
    TrackPhylogeny(),
    PurgePhylogeny(),
    ClearInteractionsAndRecords(),
    create_op("Reporter",
        retriever=Jevo.get_individuals,
        operator=(s,individuals)-> foreach(i->println(i.genotype), individuals)
    ),
    Reporter(GenotypeSum, console=true)], counters=counters)

run!(state, n_gens)


VectorGenotype(Float32[0.26209486, 0.46431565])
VectorGenotype(Float32[-0.580678, -0.90946543])
VectorGenotype(Float32[0.6229845, 0.012773156])
VectorGenotype(Float32[0.79820263, -1.1590782])
gen=1 GenotypeSum: |-1.49, -0.122 ± 1.036, 0.726|, 4 samples
VectorGenotype(Float32[0.26209486, 0.46431565])
VectorGenotype(Float32[0.3217729, 0.65762794])
VectorGenotype(Float32[0.6229845, 0.012773156])
VectorGenotype(Float32[0.6152732, 0.3734604])
gen=2 GenotypeSum: |0.636, 0.833 ± 0.179, 0.989|, 4 samples
VectorGenotype(Float32[0.26209486, 0.46431565])
VectorGenotype(Float32[-0.36359048, 1.8665471])
VectorGenotype(Float32[0.6229845, 0.012773156])
VectorGenotype(Float32[-0.7124732, -1.0165223])
gen=3 GenotypeSum: |-1.729, 0.284 ± 1.397, 1.503|, 4 samples
VectorGenotype(Float32[0.26209486, 0.46431565])
VectorGenotype(Float32[0.3374526, 1.1012497])
VectorGenotype(Float32[0.6229845, 0.012773156])
VectorGenotype(Float32[-0.5705619, 0.39416242])
gen=4 GenotypeSum: |-0.176, 0.656 ± 0.661, 1.439|, 4 sa

# Viewing the Phylogeny

Jevo outputs phylogenies compliant with the [ALIFE Data Standards](https://github.com/alife-data-standards/alife-data-standards) phylogeny format. Each population outputs a separate phylogeny file. For our co-evolutionary setup above, we output two phylogenies:

In [ ]:
;head p1-phylo.csv p2-phylo.csv

==> p1-phylo.csv <==
id, ancestor_list
1,
2,
5, 1
7, 1
9, 1
11, 1
13, 1
15, 1
17, 1

==> p2-phylo.csv <==
id, ancestor_list
3,
4,
6, 3
8, 3
10, 3
12, 3
14, 3
16, 3
18, 3


In [ ]:
;pip install alifedata-phyloinformatics-convert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 802.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.5/459.5 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 50.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.6/245.6 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.7 MB/s eta 0:00:00
  Using cached lyncs_setuptools-0.3.2-py3-none-any.whl
  Using cached GitPython-3.

In [ ]:
# Since we are in a julia interpreter, we need to write and execute python code from the command line
open("convert.py", "w") do f
  println(f, """
  import io
  import pathlib

  import alifedata_phyloinformatics_convert as apc
  import pandas

  p1_phylo = pandas.read_csv("p1-phylo.csv")
  p2_phylo = pandas.read_csv("p2-phylo.csv")

  converter = apc.RosettaTree(p1_phylo)
  converter = apc.RosettaTree(p2_phylo)
  """)
end

In [ ]:
;python convert.py

Traceback (most recent call last):
  File "/content/convert.py", line 10, in <module>
    converter = apc.RosettaTree(p1_phylo)
  File "/usr/local/lib/python3.10/dist-packages/alifedata_phyloinformatics_convert/RosettaTree.py", line 121, in __init__
    raise ValueError(
ValueError: Unsupported tree format tree=    id   ancestor_list
0    1             NaN
1    2             NaN
2    5             1.0
3    7             1.0
4    9             1.0
5   11             1.0
6   13             1.0
7   15             1.0
8   17             1.0
9   19             1.0
10  21            19.0
11  23            19.0 of type <class 'pandas.core.frame.DataFrame'>
